# Zillow Challenge

##  Data input

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import seterr,isneginf,array
from datetime import datetime
from pandas import compat
from operator import itemgetter
from sklearn import tree
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.neighbors import KDTree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from IPython.display import Image

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 200)
pd.options.display.float_format = '{:20,.2f}'.format
compat.PY3 = False

def Decision_Tree_Image(decision_tree, feature_names, name="temp"):
    
    # Etrainport our decision tree to graphviz format
    dot_file = tree.export_graphviz(decision_tree, out_file='images/' + name + '.dot', feature_names=feature_names)
    
    # Call graphviz to make an image file from our decision tree
    os.system("dot -T png images/" + name + ".dot -o images/" + name + ".png")
    
    # Return the .png image so we can see it
    return Image(filename='images/' + name + '.png')

In [2]:
train_f = "/Users/Jose/Desktop/Zillow/train_2016_v2.csv"
train = pd.read_csv(train_f, index_col = 'parcelid', parse_dates=['transactiondate'])
train['transactiondate'] = pd.to_datetime(train['transactiondate']).astype(int)

properties_f = "/Users/Jose/Desktop/Zillow/properties_2016.csv"
properties = pd.read_csv(properties_f, index_col = 'parcelid')


train = train.join(properties) 
train_bak = train  # backup 

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#### TEST MODE  #####
train = train_bak
train = train.sample(frac=.2)

##  Feature Analysis

In [167]:
features_complete = train.columns[list(~train.isnull().any().values)]
features_complete

Index([u'logerror', u'transactiondate', u'bathroomcnt', u'bedroomcnt', u'fips',
       u'latitude', u'longitude', u'propertycountylandusecode',
       u'propertylandusetypeid', u'rawcensustractandblock', u'regionidcounty',
       u'roomcnt', u'assessmentyear'],
      dtype='object')

In [168]:
train.dtypes

logerror                        float64
transactiondate                   int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                   object
heatingorsystemtypeid           float64


##  Feature creation

In [45]:
features_created = set()

In [46]:
train.isnull().sum()

logerror                            0
transactiondate                     0
airconditioningtypeid           12416
architecturalstyletypeid        18004
basementsqft                    18049
bathroomcnt                         0
bedroomcnt                          0
buildingclasstypeid             18054
buildingqualitytypeid            6705
calculatedbathnbr                 231
decktypeid                      17901
finishedfloor1squarefeet        16686
calculatedfinishedsquarefeet      128
finishedsquarefeet12              939
finishedsquarefeet13            18050
finishedsquarefeet15            17340
finishedsquarefeet50            16686
finishedsquarefeet6             17964
fips                                0
fireplacecnt                    16097
fullbathcnt                       231
garagecarcnt                    11954
garagetotalsqft                 11954
hashottuborspa                  17562
heatingorsystemtypeid            6928
latitude                            0
longitude   

In [171]:
#################
#  regionidzip_avg
#################

def add_logerror_byregionidzip(df, cache):
    df = df.join(cache,on='regionidzip', rsuffix='_byregionidzip')

# create during execution
features_created.add('regionidzip_avg')
features_created.add('regionidzip')

In [172]:
#################
#  finishedsquarefeet[n]_t
#################

def add_finishedsquarefeet(df, feature):
    n = 100
    df[feature+'_t'] = np.floor(np.divide(df[feature], n))
    df[feature] = df[feature].fillna(-1)
    df[feature+'_t'] = df[feature+'_t'].fillna(-1)
    
def add_finishedsquarefeet12_t(df):
    add_finishedsquarefeet(df, 'finishedsquarefeet12')
def add_finishedsquarefeet13_t(df):
    add_finishedsquarefeet(df, 'finishedsquarefeet13')
def add_finishedsquarefeet15_t(df):
    add_finishedsquarefeet(df, 'finishedsquarefeet15')    
def add_finishedsquarefeet50_t(df):
    add_finishedsquarefeet(df, 'finishedsquarefeet50')
def add_finishedsquarefeet6_t(df):
    add_finishedsquarefeet(df, 'finishedsquarefeet6')

features_created.add('finishedsquarefeet12_t')
features_created.add('finishedsquarefeet13_t')
features_created.add('finishedsquarefeet15_t')
features_created.add('finishedsquarefeet50_t')
features_created.add('finishedsquarefeet6_t')

#Needed for future calculations, so keeping for now
features_created.add('finishedsquarefeet12')
features_created.add('finishedsquarefeet13')
features_created.add('finishedsquarefeet15')
features_created.add('finishedsquarefeet50')
features_created.add('finishedsquarefeet6')

In [4]:
train['taxrate'] = np.divide(train['taxamount'],train['taxvaluedollarcnt'])

In [6]:
train['taxrate'].describe()

count              18,054.00
mean                    0.02
std                     0.82
min                     0.00
25%                     0.01
50%                     0.01
75%                     0.01
max                   109.54
Name: taxrate, dtype: float64

In [44]:
train.query('logerror<-2 & logerror > -2.5')
#train.query('taxrate==.01')

,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,taxrate
parcelid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14621371,-2.27,1460505600000000000,nan,nan,nan,2.00,2.00,nan,nan,2.00,nan,nan,"1,335.00","1,335.00",nan,nan,nan,nan,"6,059.00",nan,2.00,nan,nan,NaN,24.00,"33,615,100.00","-117,719,000.00",nan,nan,nan,nan,nan,nan,34,266.00,NaN,"60,590,626.22","46,098.00","1,286.00",nan,"96,963.00",5.00,nan,nan,nan,nan,nan,nan,"1,969.00",1.00,NaN,"123,518.00","234,595.00","2,015.00","111,077.00","2,468.84",NaN,nan,"60,590,626,222,004.00",0.01
14426239,-2.32,1465948800000000000,nan,nan,nan,2.50,4.00,nan,nan,2.50,nan,nan,"2,739.00","2,739.00",nan,nan,nan,nan,"6,059.00",nan,2.00,2.00,632.00,NaN,nan,"33,479,443.00","-117,683,221.00","6,002.00",nan,nan,nan,nan,nan,122,261.00,NaN,"60,590,423.11","17,686.00","1,286.00",nan,"96,961.00",8.00,nan,1.00,nan,nan,nan,nan,"1,978.00",2.00,NaN,"237,532.00","830,000.00","2,015.00","592,468.00","9,399.26",NaN,nan,"60,590,423,106,004.00",0.01
14470669,-2.31,1470873600000000000,nan,nan,nan,2.50,4.00,nan,nan,2.50,nan,nan,"2,373.00","2,373.00",nan,nan,nan,nan,"6,059.00",nan,2.00,2.00,438.00,True,nan,"33,576,718.00","-117,654,172.00","5,454.00",1.00,nan,nan,1.00,nan,122,261.00,NaN,"60,590,320.40","12,773.00","1,286.00",nan,"96,996.00",0.00,nan,1.00,nan,nan,nan,nan,"1,986.00",nan,NaN,"229,283.00","653,189.00","2,015.00","423,906.00","7,755.82",NaN,nan,"60,590,320,402,000.00",0.01
13989753,-2.35,1464825600000000000,nan,nan,nan,2.00,4.00,nan,nan,2.00,nan,nan,"1,053.00","1,053.00",nan,nan,nan,nan,"6,059.00",nan,2.00,1.00,396.00,NaN,nan,"33,823,340.00","-117,960,906.00","7,260.00",nan,nan,nan,nan,nan,122,261.00,NaN,"60,590,877.01","16,764.00","1,286.00","275,496.00","97,023.00",7.00,nan,nan,nan,nan,nan,nan,"1,955.00",1.00,NaN,"27,036.00","52,055.00","2,015.00","25,019.00",983.62,NaN,nan,"60,590,877,012,002.00",0.02
14334592,-2.29,1465776000000000000,nan,nan,nan,2.00,3.00,nan,nan,2.00,nan,nan,"2,029.00","2,029.00",nan,nan,nan,nan,"6,059.00",nan,2.00,2.00,400.00,NaN,nan,"33,761,934.00","-117,793,829.00","12,257.00",1.00,nan,nan,nan,1.00,122,261.00,NaN,"60,590,756.06","40,081.00","1,286.00",nan,"97,005.00",7.00,nan,nan,nan,nan,nan,nan,"1,960.00",1.00,NaN,"125,328.00","623,987.00","2,015.00","498,659.00","7,179.28",NaN,nan,"60,590,756,063,001.00",0.01
11839942,-2.32,1469145600000000000,1.00,nan,nan,3.00,2.00,nan,4.00,3.00,nan,nan,"1,409.00","1,409.00",nan,nan,nan,nan,"6,037.00",nan,3.00,nan,nan,NaN,2.00,"34,063,100.00","-118,155,000.00",985.00,nan,nan,nan,nan,nan,010D,269.00,ALRPD*,"60,374,819.02","50,677.00","3,101.00",nan,"96,533.00",0.00,nan,nan,nan,1.00,nan,nan,"1,980.00",nan,NaN,"173,471.00","375,015.00","2,015.00","201,544.00","4,619.23",NaN,nan,"60,374,819,021,029.00",0.01
14648465,-2.19,1460419200000000000,nan,nan,nan,2.00,2.00,nan,nan,2.00,nan,nan,"1,188.00","1,188.00",nan,nan,nan,nan,"6,059.00",nan,2.00,1.00,0.00,NaN,24.00,"33,608,700.00","-117,746,000.00",nan,nan,nan,nan,nan,nan,34,266.00,NaN,"60,590,626.49","46,098.00","1,286.00",nan,"96,963.00",5.00,nan,nan,nan,nan,nan,nan,"1,973.

In [47]:
# Univariate - float

feature = 'roomcnt'
plt.xlabel(feature)
plt.ylabel('logerror')
plt.scatter(train[feature], 
            train['logerror'],
            c=train['logerror'],
            s=20, lw=0,vmin=-4, vmax=4,
           cmap='seismic')
plt.axis([ #0,.06,
        train[feature].min(), 
          train[feature].max(),
          train['logerror'].min(), 
            train['logerror'].max()
         ])

plt.show()

In [35]:
# Bivariate
feature1 = 'propertylandusetypeid'
feature2 = 'taxamount'


plt.xlabel(feature1)
plt.ylabel(feature2)
plt.scatter(train[feature1], 
            train[feature2],
            c=train['logerror'],
            s=20, lw=0,
           cmap='seismic',vmin=-4, vmax=4)
plt.axis([
        #train[feature1].min(), 
        240,
        train[feature1].max(),
        #      95000,99000, 0, .1,
             train[feature2].min(),  train[feature2].max(),
             #0,.1
         ])
plt.show()

Conclusions:
   * Average logerror = 0.011457219606756575
   * In general, data usually underestimates, with the etrainception of fewer high overestimates.
   * totalroomcnt:  ==0 is a mitrained bag.  != zero is overestimating

##  Feature selection

In [173]:
features_complete

Index([u'logerror', u'transactiondate', u'bathroomcnt', u'bedroomcnt', u'fips',
       u'latitude', u'longitude', u'propertycountylandusecode',
       u'propertylandusetypeid', u'rawcensustractandblock', u'regionidcounty',
       u'roomcnt', u'assessmentyear'],
      dtype='object')

In [174]:
features_to_keep = features_complete 
features_to_keep = features_to_keep.union(features_created)

In [175]:
features_to_keep

Index([u'assessmentyear', u'bathroomcnt', u'bedroomcnt',
       u'finishedsquarefeet12', u'finishedsquarefeet12_t',
       u'finishedsquarefeet13', u'finishedsquarefeet13_t',
       u'finishedsquarefeet15', u'finishedsquarefeet15_t',
       u'finishedsquarefeet50', u'finishedsquarefeet50_t',
       u'finishedsquarefeet6', u'finishedsquarefeet6_t', u'fips', u'latitude',
       u'logerror', u'longitude', u'propertycountylandusecode',
       u'propertylandusetypeid', u'rawcensustractandblock', u'regionidcounty',
       u'regionidzip', u'regionidzip_avg', u'roomcnt', u'transactiondate'],
      dtype='object')

In [157]:
Y = train[['logerror']]
X = train
for column in X.columns:
    if column not in features_to_keep:
        X = X.drop(column, axis=1)
        
#X = pd.DataFrame(scale(train, axis=0, with_mean=True, with_std=True, copy=True), columns = X.columns.values)

In [158]:
# TODO: qualifiers
X = X.drop('propertylandusetypeid', axis =1)
#X = X.drop('rawcensustractandblock', axis =1)
#X = X.drop('regionidcounty', axis =1)

In [159]:
X.describe()

,logerror,transactiondate,bathroomcnt,bedroomcnt,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,latitude,longitude,regionidzip,roomcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt
count,"45,138.00","45,138.00","45,138.00","45,138.00","42,769.00",18.00,"1,808.00","3,431.00",199.00,"45,138.00","45,138.00","45,138.00","45,124.00","45,138.00","45,138.00","45,138.00","45,138.00"
mean,0.01,"1,465,632,530,214,005,248.00",2.28,3.04,"1,748.58","1,407.00","2,389.88","1,351.42","2,307.68","6,048.82","34,006,793.52","-118,199,189.42","96,596.67",1.48,"459,704.20","2,015.00","279,624.96"
std,0.16,"7,434,938,733,608,052.00",1.00,1.16,906.44,90.09,"1,099.09",658.94,"1,289.41",20.66,"265,460.80","359,915.30","4,056.91",2.82,"569,459.15",0.00,"413,006.66"
min,-2.40,"1,451,606,400,000,000,000.00",0.00,0.00,2.00,"1,248.00",609.00,49.00,438.00,"6,037.00","33,339,295.00","-119,447,865.00","95,982.00",0.00,"7,704.00","2,015.00","1,319.00"
25%,-0.03,"1,459,728,000,000,000,000.00",2.00,2.00,"1,178.00","1,356.00","1,673.50",938.00,"1,268.00","6,037.00","33,813,046.75","-118,411,000.00","96,193.00",0.00,"198,855.25","2,015.00","81,738.00"
50%,0.01,"1,465,862,400,000,000,000.00",2.00,3.00,"1,524.00","1,440.00","2,108.50","1,250.00","2,052.00","6,037.00","34,023,673.00","-118,173,156.00","96,393.00",0.00,"342,872.00","2,015.00","192,913.00"
75%,0.04,"1,471,478,400,000,000,000.00",3.00,4.00,"2,058.00","1,440.00","2,856.00","1,624.00","3,270.00","6,059.00","34,173,100.00","-117,922,166.25","96,987.00",0.00,"541,080.75","2,015.00","345,994.25"
max,4.74,"1,483,056,000,000,000,000.00",12.00,16.00,"15,973.00","1,566.00","22,741.00","8,352.00","7,224.00","6,111.00","34,816,009.00","-117,554,924.00","399,675.00",15.00,"27,750,000.00","2,015.00","24,500,000.00"


In [160]:
Y.describe()

,logerror
count,"45,138.00"
mean,0.01
std,0.16
min,-2.40
25%,-0.03
50%,0.01
75%,0.04
max,4.74


## Data cleaning

## Modeling

In [161]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.1)

In [162]:
# Cache: regionidzip -> mean(logerror)
_byzipcache = X_train.groupby(['regionidzip'])['logerror'].mean()

In [163]:
# Create new vars needed after train set is defined
add_logerror_byregionidzip(X_train, _byzipcache )
X_train = X_train.drop('logerror', axis = 1)
X_train = X_train.drop('regionidzip', axis = 1)
add_distanceavg(train)
add_finishedsquarefeet12_t(X_train)
X_train = X_train.drop('finishedsquarefeet12', axis = 1)
add_finishedsquarefeet13_t(X_train)
X_train = X_train.drop('finishedsquarefeet13', axis = 1)
add_finishedsquarefeet15_t(X_train)
X_train = X_train.drop('finishedsquarefeet15', axis = 1)
add_finishedsquarefeet50_t(X_train)
X_train = X_train.drop('finishedsquarefeet50', axis = 1)
add_finishedsquarefeet6_t(X_train)
X_train = X_train.drop('finishedsquarefeet6', axis = 1)

KeyboardInterrupt: 

In [142]:
add_logerror_byregionidzip(X_test, _byzipcache )
X_test = X_test.drop('logerror', axis = 1)
X_test = X_test.drop('regionidzip', axis = 1)
add_distanceavg(X_test)
add_finishedsquarefeet12_t(X_test)
X_test = X_test.drop('finishedsquarefeet12', axis = 1)
add_finishedsquarefeet13_t(X_test)
X_test = X_test.drop('finishedsquarefeet13', axis = 1)
add_finishedsquarefeet15_t(X_test)
X_test = X_test.drop('finishedsquarefeet15', axis = 1)
add_finishedsquarefeet50_t(X_test)
X_test = X_test.drop('finishedsquarefeet50', axis = 1)
add_finishedsquarefeet6_t(X_test)
X_test = X_test.drop('finishedsquarefeet6', axis = 1)

In [143]:
# Fit regression model
max_depth = None
min_samples_split = int(round(len(X_train)/50, 0))
min_samples_leaf = int(round(len(X_train)/10, 0))
print("min_samples_split =", min_samples_split)
print("min_samples_split =", min_samples_leaf)

model = DecisionTreeRegressor(max_depth=max_depth, 
                              min_samples_split = min_samples_split,
                              min_samples_leaf = min_samples_leaf,
                              criterion = "mae")
model.fit(X_train, Y_train)

('min_samples_split =', 90)
('min_samples_split =', 451)


DecisionTreeRegressor(criterion='mae', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=451,
           min_samples_split=90, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [144]:
Y_predict=model.predict(X_test)

In [145]:
# (<= value) left = false, right = true
zip(X.columns[model.tree_.feature], model.tree_.threshold, model.tree_.children_left, model.tree_.children_right)

[('finishedsquarefeet6', 26.5, 1, 14),
 ('logerror', 1.4726447799960863e+18, 2, 13),
 ('finishedsquarefeet12', 34234048.0, 3, 12),
 ('logerror', 1.456401557279998e+18, 4, 5),
 ('roomcnt', -2.0, -1, -1),
 ('logerror', 1.4623631093257994e+18, 6, 7),
 ('roomcnt', -2.0, -1, -1),
 ('logerror', 1.4697072972435292e+18, 8, 11),
 ('finishedsquarefeet12', 33979432.0, 9, 10),
 ('roomcnt', -2.0, -1, -1),
 ('roomcnt', -2.0, -1, -1),
 ('roomcnt', -2.0, -1, -1),
 ('roomcnt', -2.0, -1, -1),
 ('roomcnt', -2.0, -1, -1),
 ('roomcnt', -2.0, -1, -1)]

## Model evaluation

In [146]:
# Zillow MAE (test)
print("MAE: ",metrics.mean_absolute_error(Y_test['logerror'], Y_predict))

('MAE: ', 0.068615931076923084)


## Model Execution

In [469]:
sample_submission_f = '/Users/Jose/Desktop/Zillow/sample_submission.csv'
submission = pd.read_csv(sample_submission_f, index_col='ParcelId')

In [470]:
#  ATTEMPT 2 - 
logerroravg = train['logerror'].mean()
print("Using logerroravg = ", logerroravg)

('Using logerroravg = ', 0.011457219606756525)


In [546]:
X_all = properties

In [548]:
# Add features

def add_transactiondate(df):
    df['transactiondate'] = pd.datetime(2016,10,1)
    df['transactiondate'] = df['transactiondate'].astype(int)

print("Calculating finishedsquarefeet12_t")
add_finishedsquarefeet12_t(X_all)
print("Calculating finishedsquarefeet13_t")
add_finishedsquarefeet13_t(X_all)
print("Calculating finishedsquarefeet15_t")
add_finishedsquarefeet15_t(X_all)
print("Calculating finishedsquarefeet50_t")
add_finishedsquarefeet50_t(X_all)
print("Calculating finishedsquarefeet6_t")
add_finishedsquarefeet6_t(X_all)
print("Calculating regionidzip_avg")
add_regionidzip_avg(X_all)
print("Calculating transactiondate")
add_transactiondate(X_all)

KeyboardInterrupt: 

In [ ]:
# Expensive calc
print("Calculating distanceavg")
zillow.add_distanceavg(X_all)

In [ ]:
for column in X_all.columns:
    if column not in features_to_keep:
        X_all = X_all.drop(column, axis=1)

In [547]:
#  Update data
print("Missing: ",X_test.columns.difference( X_all.columns))

('Missing: ', Index([u'distanceavg', u'finishedsquarefeet12_t', u'finishedsquarefeet13_t',
       u'finishedsquarefeet15_t', u'finishedsquarefeet50_t',
       u'finishedsquarefeet6_t', u'regionidzip_avg', u'transactiondate'],
      dtype='object'))


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  from ipykernel import kernelapp as app


In [534]:
Y_all = model.predict(X_all)

In [ ]:
results = pd.DataFrame(index=X_all.index)
results.index.names = ['ParcelId']
results['201610'] = Y_all
results['201611'] = Y_all
results['201612'] = Y_all
results['201710'] = Y_all
results['201711'] = Y_all
results['201712'] = Y_all

In [ ]:
submission = submission.drop(submission.columns[0:], axis=1)
submission = submission.join(results)

# Use average for properties with missing data
submission.fillna(logerroravg, inplace=True)       
submission.columns

In [ ]:
# Round as per rules
submission = submission.round(4)

## Sanity Checks

In [ ]:
# Average log error
submission.describe().round(4)

In [ ]:
# Check for NaN
submission.isnull().sum()

In [ ]:
# Check if any duplicates
submission[submission.index.duplicated(keep=False)]

In [ ]:
# Check additional values in submission file
submission[~submission.index.isin(properties.index)]

In [ ]:
# Check additional values in properties file
properties[~properties.index.isin(submission.index)]

## Dump File

In [ ]:
# Write file
submission_f = '/Users/Jose/Desktop/Zillow/submission.csv'
submission_fh = open(submission_f, 'wb')
submission.to_csv(submission_fh, sep=',', header='true')
submission_fh.close()

## Historical Records

## Resources

In [ ]:
# bedroomcnt
#plt.trainlabel('bedroomcnt')
#plt.ylabel('logerror')
#plt.scatter(train['bathroomcnt'], train['logerror'], 
#           alpha=1, s=2, color='r')
#plt.atrainis([0, 10,-5, 5])
#plt.show()

#date
#plt.trainlabel('transactiondate')
#plt.ylabel('logerror')
#plt.scatter(train['bedroomcnt'], train['logerror'], alpha=1, s=2, color='r')
#plt.atrainis(['2016-01-01', '2017-01-01',-5, 5])
#plt.gcf().autofmt_traindate()
#plt.show()

#import matplotlib as mpl
#plt.trainlabel('longitude')
#plt.ylabel('latitude')
#plt.scatter(train['longitude'], train['latitude'], c=train['logerror'], s=20, lw=0, cmap='seismic')
#plt.atrainis([min(train['longitude']), matrain(train['longitude']),min(train['latitude']), matrain(train['latitude'])])
#plt.colorbar()
#plt.show()

#train2 = train.query('logerror < -2.')

#train2 = train.query('roomcnt == 0 ')

#train2 = train.query('bedroomcnt > 2')

#train2 = train.sample(frac=.10)

#plt.trainlabel('longitude')
#plt.ylabel('latitude')
#plt.scatter(train2['longitude'], train2['latitude'], c=train2['logerror'], s=20, lw=0, cmap='seismic')
#plt.colorbar()
#plt.show()